In [1]:
import pandas as pd
import datetime
import glob
import numpy as np
import itertools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from tilden_fluxdata_headers_conversion import headers_3m, headers_10m
import sys
sys.path.append('../')
import sosutils

# Read one day's data

In [2]:
df_21_a = pd.read_csv('../data/flux_data_original_from_tilden/KPA21_310-365.FLX', delim_whitespace=True)
df_21_b = pd.read_csv('../data/flux_data_original_from_tilden/KPB21_310-365.FLX', delim_whitespace=True)
df_22_a = pd.read_csv('../data/flux_data_original_from_tilden/KPA22_001-270.FLX', delim_whitespace=True)
df_22_b = pd.read_csv('../data/flux_data_original_from_tilden/KPB22_001-270.FLX', delim_whitespace=True)

In [3]:
df_21_a['year'] = 2021
df_21_b['year'] = 2021
df_21_a.loc[df_21_a['DecDay']==1.0, 'year'] = 2022
df_21_b.loc[df_21_b['DecDay']==1.0, 'year'] = 2022

df_22_a['year'] = 2022
df_22_b['year'] = 2022

Tall Tower (Western Tower, 10m), Instrumentation
===
* Prop vane anemometer (3m, 5m, 10m)  - MET DATA
* 2 independent temperature measurements (3m, 5m, 10m)
* Moisture/RH sensors (5m)
* Sonic Anemometer (10m)
* 3 independent IR senors pointed at ground
* Water vapor and CO2 flux (10m)
* Radiation in and out, longwave and shortwave (~4m)



'df_21_a.columns'
===
'DecDay',

'Day',

'HHMM',

'H_10m', - sensible heat flux (10m)

'qc_H', - sensible heat flux (10m) QC flag

'LE_10m', - latent heat flux (10m)

'qc_LE', - latent heat flux (10m) QC flag

'co2_flx_10m', 

'co2_mix_ratio',

'Ts_10m', - sonic temperature 

'Tair_10m', - sonic temp corrected to air temp

'pressure', - atmospheric pressure

'density', - atmospheric density?

'e_10m', - water vapor pressure

'es_10m', - saturated water vapor pressure

'RH', - relative humidity (10m ???)

'Tdew', - dew point temp (???)

'u_unrot_10m',

'v_unrot_10m',

'w_unrot_10m',

'u_rot_10m',

'wind_speed', - calculated with the sonices 

'max_wind_speed',

'wind_dir',

'u*_10m',

'TKE_10m',

'u_var_10m', - is this u' or u variance over the 30 minute averaging period ASK HOW THESE ARE CALCULATED - what reynolds averaging time period was used

'v_var_10m',

'w_var_10m',

'Ts_var_10m',

'co2_var_10m',

'h2o_var_10m'

In [4]:
df_21_a = df_21_a.rename(columns=headers_10m)
df_21_b = df_21_b.rename(columns=headers_3m)
df_22_a = df_22_a.rename(columns=headers_10m)
df_22_b = df_22_b.rename(columns=headers_3m)

In [25]:
df_22_b['day'].apply(float)

ValueError: could not convert string to float: '00:30:00'

In [13]:
for df in [df_21_a, df_21_b, df_22_a, df_22_b]:
    df['day'] = df['day'].astype('int')
    df['hour_minute'] = df['hour_minute'].astype('int')

ValueError: invalid literal for int() with base 10: '1.0'

In [8]:
df_21_a['date'] = df_21_a.apply(
    lambda row: sosutils.time_from_day_and_hhmm(
        row['day'], 
        row['hour_minute'], 
        base_day = datetime.datetime(int(row['year']), 1, 1)
    ), 
    axis='columns'
)

df_21_b['date'] = df_21_b.apply(
    lambda row: sosutils.time_from_day_and_hhmm(
        row['day'], 
        row['hour_minute'], 
        base_day = datetime.datetime(int(row['year']), 1, 1)
    ), 
    axis='columns'
)


df_22_a['date'] = df_22_a.apply(
    lambda row: sosutils.time_from_day_and_hhmm(
        row['day'], 
        row['hour_minute'], 
        base_day = datetime.datetime(int(row['year']), 1, 1)
    ), 
    axis='columns'
)

df_22_b['date'] = df_22_b.apply(
    lambda row: sosutils.time_from_day_and_hhmm(
        row['day'], 
        row['hour_minute'], 
        base_day = datetime.datetime(int(row['year']), 1, 1)
    ), 
    axis='columns'
)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
df_21_a['height'] = 10
df_21_b['height'] = 3
df_22_a['height'] = 10
df_22_b['height'] = 3

In [70]:
with_height_cols = [col for col in list(df_21_a.columns) if '_10m' in col]
without_height_cols = [f.replace('_10m', '') for f in with_height_cols]
df_21_a = df_21_a.rename(columns=dict(zip(with_height_cols, without_height_cols)))

with_height_cols = [col for col in list(df_22_a.columns) if '_10m' in col]
without_height_cols = [f.replace('_10m', '') for f in with_height_cols]
df_22_a = df_22_a.rename(columns=dict(zip(with_height_cols, without_height_cols)))

with_height_cols = [col for col in list(df_21_b.columns) if '_3m' in col]
without_height_cols = [f.replace('_3m', '') for f in with_height_cols]
df_21_b = df_21_b.rename(columns=dict(zip(with_height_cols, without_height_cols)))

with_height_cols = [col for col in list(df_22_b.columns) if '_3m' in col]
without_height_cols = [f.replace('_3m', '') for f in with_height_cols]
df_22_b = df_22_b.rename(columns=dict(zip(with_height_cols, without_height_cols)))

In [71]:
df = pd.concat([
    df_21_a, df_21_b, df_22_a, df_22_b
])

In [72]:
df.to_parquet('../data/tilden_fluxdata_consolidated.parquet')

In [73]:
df_22_a.head()

,Decday,day,hour_minute,H,LE,co2_flx,co2_mix_ratio,Ts,Tair,pressure,density,e,es,RH,Tdew,u_unrot,v_unrot,w_unrot,u_rot,wind_speed,max_wind_speed,wind_dir,u*,TKE,u_var,v_var,w_var,Ts_var,co2_var,h2o_var,year,date,height
0,1.0208,1,30,-3.1,-0.6,0.234,397.3,263.88,263.25,70251.3,0.928,243.82,287.88,84.70,261.13,-1.782,0.979,0.000,2.033,2.033,3.654,299.2,0.149,0.301,0.32185,0.27187,0.01061,0.09550,0.00221,1.22801,2022,2022-01-01 00:30:00,10
1,1.0416,1,100,-4.6,-2.3,-0.194,397.6,263.70,263.06,70251.7,0.929,234.28,283.65,82.59,260.64,-2.272,0.485,0.048,2.324,2.324,4.079,281.5,0.096,0.334,0.27970,0.35862,0.03590,0.05207,0.00187,7.71438,2022,2022-01-01 01:00:00,10
2,1.0624,1,130,-4.3,-0.6,-0.674,396.1,263.63,263.16,70259.5,0.929,228.15,285.93,79.79,260.31,-1.475,0.412,0.065,1.533,1.533,3.272,285.0,0.149,0.376,0.41006,0.30159,0.04795,0.03470,0.00123,1.45999,2022,2022-01-01 01:30:00,10
3,1.0833,1,200,-6.6,-0.2,0.143,395.7,263.40,262.78,70257.8,0.930,227.95,277.44,82.16,260.30,-2.173,1.161,0.012,2.464,2.464,4.072,297.6,0.199,0.311,0.15051,0.44574,0.03083,0.09792,0.00088,3.38138,2022,2022-01-01 02:00:00,10
4,1.1041,1,230,-14.2,-5.1,0.461,395.9,262.62,261.64,70241.1,0.934,211.28,253.40,83.38,259.37,-2.392,1.102,0.011,2.633,2.633,4.489,295.2,0.104,0.131,0.13602,0.10367,0.02712,0.16634,0.00199,5.48351,2022,2022-01-01 02:30:00,10


In [74]:
df_22_b.head()

,decimal_day,day,hour_minute,H,qc_H,LE,qc_LE,co2_flx,co2_mix_ratio,Ts,Tair,pressure,density,e,es,RH,Tdew,u_unrot,v_unrot,w_unrot,u_rot,wind_speed,max_wind_speed,wind_dir,u*,TKE,u_var,v_var,w_var,Ts_var,co2_var,h2o_var,year,date,height
0,1.021,1,30,-11.7,1,-2.9,1,1.706,410.4,262.86,262.61,70321.0,0.932,261.04,273.75,95.36,261.98,-1.033,1.241,-0.066,1.617,1.617,2.980,318.5,0.141,0.314,0.22541,0.39312,0.01222,0.22692,0.00592,5.18978,2022,2022-01-01 00:30:00,3
1,1.042,1,100,4.8,2,0.3,2,-0.835,409.5,262.95,262.65,70321.1,0.931,256.40,274.49,93.41,261.76,-1.726,0.747,-0.014,1.881,1.881,3.904,293.5,0.190,0.305,0.24465,0.34237,0.02784,0.16706,0.00566,4.93172,2022,2022-01-01 01:00:00,3
2,1.062,1,130,-0.1,2,1.4,2,-0.825,408.0,262.99,262.98,70333.1,0.930,252.91,281.78,89.75,261.59,-0.935,0.540,-0.017,1.080,1.080,3.040,302.7,0.131,0.321,0.39762,0.22454,0.02413,0.06316,0.00191,1.61498,2022,2022-01-01 01:30:00,3
3,1.083,1,200,-9.1,0,-2.0,1,0.817,408.0,262.52,262.36,70329.6,0.933,248.41,268.39,92.55,261.36,-1.505,1.354,-0.065,2.025,2.025,3.696,311.1,0.123,0.242,0.14087,0.32683,0.01884,0.26179,0.00347,8.56018,2022,2022-01-01 02:00:00,3
4,1.104,1,230,-16.3,0,-8.6,0,1.312,409.3,261.01,260.48,70311.3,0.939,221.45,230.76,95.97,259.94,-1.350,1.646,-0.089,2.131,2.131,3.361,321.2,0.073,0.185,0.11441,0.23716,0.02218,0.55729,0.00582,19.46990,2022,2022-01-01 02:30:00,3
